In [4]:
# Importing necessary libraries
%pip install pymongo[srv]
from pymongo import MongoClient
import os
import datetime
import pprint
import requests
import pymongo
import time
from datetime import datetime, timedelta
import pandas as pd

# MongoDB connection details
atlas_cluster_name = "sandbox.zibbf"
atlas_default_dbname = "local"
atlas_user_name = "jtupitza"
atlas_password = "Passw0rd1234"

# Connect to MongoDB
conn_str = f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}.mongodb.net/{atlas_default_dbname}"
client = pymongo.MongoClient(conn_str)
db = client['nfl_data']  # Choose your database name

# API URL
api_url = "https://4feaquhyai.execute-api.us-east-1.amazonaws.com/api/pi"

# Function to fetch data from API and store in MongoDB
def fetch_and_store_data():
    try:
        response = requests.get(api_url)
        data = response.json()
        # Add timestamp to data
        data['timestamp'] = datetime.now()
        # Insert data into MongoDB
        collection = db['nfl_data']  # Choose your collection name
        collection.insert_one(data)
        print("Data inserted into MongoDB at", datetime.now())
    except Exception as e:
        print("Error fetching or storing data:", e)

# Function to perform data analysis
def analyze_data():
    try:
        # Retrieve data from MongoDB
        collection = db['nfl_data']  # Choose your collection name
        cursor = collection.find({})
        data = list(cursor)
        if len(data) == 0:
            print("No data available for analysis.")
            return

        # Convert data to DataFrame for analysis
        df = pd.DataFrame(data)

        # Check for required columns
        required_columns = ['Yards.Gained', 'Touchdown', 'down', 'GoalToGo', 'posteam', 'DefensiveTeam',
                            'PlayType', 'FieldGoalResult', 'Safety', 'Timeout_Indicator', 'Season']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print("Error: Missing columns in the dataset:", missing_columns)
            return

        # Describe trends over time for specific fields
        print("Trends Over Time:")

        # Number of Touchdowns per Season
        print("\nNumber of Touchdowns per Season:")
        touchdowns_by_season = df[df['Touchdown'] == 1].groupby('Season').size()
        print(touchdowns_by_season)
        # Plot number of touchdowns per season
        plt.figure(figsize=(10, 6))
        touchdowns_by_season.plot(kind='bar', color='lightgreen')
        plt.title('Number of Touchdowns per Season')
        plt.xlabel('Season')
        plt.ylabel('Number of Touchdowns')
        plt.xticks(rotation=45)
        plt.show()

        # Play Types Analysis
        print("\nPlay Types Analysis:")
        play_types = df['PlayType'].value_counts()
        print(play_types)
        # Plot play types
        plt.figure(figsize=(12, 8))
        play_types.plot(kind='bar', color='salmon')
        plt.title('Play Types Distribution')
        plt.xlabel('Play Type')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45, ha='right')
        plt.show()

        # Field Goal Success Rate
        print("\nField Goal Success Rate:")
        field_goals_made = df[df['FieldGoalResult'] == 'Good'].shape[0]
        field_goals_attempted = df[df['FieldGoalResult'].notnull()].shape[0]
        field_goal_success_rate = (field_goals_made / field_goals_attempted) * 100 if field_goals_attempted != 0 else 0
        print(f"{field_goal_success_rate:.2f}%")

        # Safety Occurrences
        print("\nSafety Occurrences:")
        safety_count = df[df['Safety'] == 1].shape[0]
        print(safety_count)

        # Timeout Analysis
        print("\nTimeout Analysis:")
        timeout_counts = df['Timeout_Indicator'].value_counts()
        print(timeout_counts)
        # Plot timeout indicators
        plt.figure(figsize=(8, 6))
        timeout_counts.plot(kind='bar', color='lightblue')
        plt.title('Timeout Indicators')
        plt.xlabel('Timeout Indicator')
        plt.ylabel('Count')
        plt.xticks(rotation=0)
        plt.show()

        # Average Yards Gained per Play Type
        print("\nAverage Yards Gained per Play Type:")
        avg_yards_by_play_type = df.groupby('PlayType')['Yards.Gained'].mean().sort_values(ascending=False)
        print(avg_yards_by_play_type)
        # Plot average yards gained per play type
        plt.figure(figsize=(12, 8))
        avg_yards_by_play_type.plot(kind='bar', color='orange')
        plt.title('Average Yards Gained per Play Type')
        plt.xlabel('Play Type')
        plt.ylabel('Average Yards Gained')
        plt.xticks(rotation=45, ha='right')
        plt.show()

        # Most Frequent Defensive Teams
        print("\nMost Frequent Defensive Teams:")
        top_defensive_teams = df['DefensiveTeam'].value_counts().head(10)
        print(top_defensive_teams)
        # Plot most frequent defensive teams
        plt.figure(figsize=(10, 6))
        top_defensive_teams.plot(kind='bar', color='lightcoral')
        plt.title('Top 10 Most Frequent Defensive Teams')
        plt.xlabel('Defensive Team')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45)
        plt.show()

        # Explanation of changes
        print("\nExplanation of Changes:")
        print("The number of touchdowns per season provides insights into the scoring trends over the years.")
        print("Play types distribution reveals the variety of strategies teams employ during games.")
        print("Field goal success rate shows how well teams convert field goal attempts.")
        print("Safety occurrences and timeout indicators provide additional context about game situations and strategies.")
        print("Average yards gained per play type gives an idea of the effectiveness of different play strategies.")
        print("The most frequent defensive teams highlight the teams that appear most often in the dataset.")

    except Exception as e:
        print("Error analyzing data:", e)

# Main function to run the process for one hour
def main():
    # Set start time to the next minute
    start_time = datetime.now() + timedelta(minutes=1)
    start_time = start_time.replace(second=0, microsecond=0)

    # Run for 60 minutes
    end_time = start_time + timedelta(minutes=60)

    while datetime.now() < end_time:
        fetch_and_store_data()
        time.sleep(60)  # Wait for 60 seconds until next iteration

    # Perform analysis after running for 1 hour
    analyze_data()

if __name__ == "__main__":
    main()


Data inserted into MongoDB at 2024-05-10 03:38:20.658341
Data inserted into MongoDB at 2024-05-10 03:39:21.025365
Data inserted into MongoDB at 2024-05-10 03:40:21.451491
Data inserted into MongoDB at 2024-05-10 03:41:21.858958
Data inserted into MongoDB at 2024-05-10 03:42:22.329200
Data inserted into MongoDB at 2024-05-10 03:43:22.759862
Data inserted into MongoDB at 2024-05-10 03:44:23.185585
Data inserted into MongoDB at 2024-05-10 03:45:23.705398
Data inserted into MongoDB at 2024-05-10 03:46:24.148919
Data inserted into MongoDB at 2024-05-10 03:47:24.639771
Data inserted into MongoDB at 2024-05-10 03:48:25.258320
Data inserted into MongoDB at 2024-05-10 03:49:25.736904
Data inserted into MongoDB at 2024-05-10 03:50:26.391194
Data inserted into MongoDB at 2024-05-10 03:51:26.997779
Data inserted into MongoDB at 2024-05-10 03:52:27.616812
Data inserted into MongoDB at 2024-05-10 03:53:28.238825
Data inserted into MongoDB at 2024-05-10 03:54:28.916361
Data inserted into MongoDB at 2